In [1]:
from itertools import chain

import sys
import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV


In [2]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import json
from copy import deepcopy as cp
import numpy as np
from numpy import linalg as mt
from pinyin import get as ctp

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from unidecode import unidecode as und
import operator
from sklearn.model_selection import train_test_split


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## Let's use Word2Vec data to build a Chinese NER system

 We use Chinese data.

# Load training and validation Data

In [4]:
def readdata(path):    
    with open(path, 'r',encoding="utf-8") as f:
     	out= json.load(f)
    return out

In [5]:
%%time
train=readdata('train_char.json')
validation=readdata('validation_char.json')

Wall time: 1.13 s


This is the first sentence in the Data

In [8]:
train[0]

[['記', 'VK', 'O'],
 ['得', 'VK', 'O'],
 ['小', 'Nd', 'B_Time'],
 ['時', 'Nd', 'I_Time'],
 ['候', 'Nd', 'I_Time'],
 ['，', 'COMMACATEGORY', 'O'],
 ['媽', 'Na', 'B_Person'],
 ['媽', 'Na', 'I_Person'],
 ['說', 'VE', 'O'],
 ['起', 'VE', 'O'],
 ['哪', 'Nep', 'O'],
 ['個', 'Nf', 'O'],
 ['典', 'VH', 'O'],
 ['型', 'VH', 'O'],
 ['敗', 'Na', 'B_Person'],
 ['家', 'Na', 'I_Person'],
 ['子', 'Na', 'I_Person'],
 ['形', 'Na', 'O'],
 ['象', 'Na', 'O'],
 ['，', 'COMMACATEGORY', 'O'],
 ['掛', 'VC', 'O'],
 ['在', 'P', 'O'],
 ['嘴', 'Na', 'O'],
 ['邊', 'Ncd', 'O'],
 ['的', 'DE', 'O'],
 ['就', 'Cbb', 'O'],
 ['是', 'Cbb', 'O'],
 ['那', 'Nep', 'B_Person'],
 ['人', 'Na', 'I_Person'],
 ['吃', 'VC', 'O'],
 ['喝', 'VC', 'O'],
 ['嫖', 'Na', 'O'],
 ['賭', 'Na', 'O'],
 ['癮', 'Na', 'O'],
 ['五', 'Na', 'O'],
 ['毒', 'Na', 'O'],
 ['俱', 'VH', 'O'],
 ['全', 'VH', 'O'],
 ['。', 'PERIODCATEGORY', 'O']]

# Word2Vec

In [10]:
%%time
Data=cp(train)
for x in validation:
    Data.append(x)

Wall time: 3.78 s


In [75]:
%%time
word2vec1 = KeyedVectors.load_word2vec_format("ch.300.bin", binary=True)
word2vec2= KeyedVectors.load_word2vec_format("cna.word2vec.bin", binary=True)



Wall time: 13.9 s


In [76]:
def get_dict(word2vec):
    Dict=dict()
    for x in word2vec.vocab:
        if x not in Dict:
           Dict[x]=word2vec.wv[x]
    return Dict 


In [77]:
%%time
Dict_wv_1=get_dict(word2vec1)
Dict_wv_2=get_dict(word2vec2)

C:\Users\oliver\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


Wall time: 1.8 s


In [20]:
def Ac_dict(train_word,Dict):
    cDict=dict()
    for x in train_word:
      for j in x:
        if j[0] in Dict:
         if j[0] not in cDict:
            cDict[j[0]]= Dict[j[0]]
        else:
             cDict[j[0]]=np.zeros(len(Dict['是']))
    return  cDict 

In [21]:
%%time
DDict=Ac_dict(Data,Dict_wv_1)
DDict2=Ac_dict(Data,Dict_wv_1)

Wall time: 261 ms


In [74]:
DDict['是'][:10]

array([ 0.3976251 ,  0.35810676,  0.13722947,  0.07472387, -0.5475522 ,
        0.41480562, -0.11194973,  0.05129677, -0.5257993 ,  0.20008406],
      dtype=float32)

## Features 1
Refer to sample code method: part-of-speech, upper / lower / titlecase, suﬃx, word length...

In [25]:
def word2features(sent, i):  
    word = sent[i][0]  
    postag = sent[i][1]  
    
    features = {  

        'bias': 1.0,  

        'word.lower()': word,  
        'word.istitle()': word.istitle(),  
        'word.isdigit()': word.isdigit(),  
        'word.isnumeric()': word.isnumeric(),   
        
        'postag': postag,  
        'word.istitle()': postag.istitle(), 
        'postag[:2]': postag[:2],   
    }  
 
    if i > 0:  
  
        word1 = sent[i-1][0]  
        postag1 = sent[i-1][1]    
        features.update({  
            
            '-1:word.lower()': word1, 
            '-1:postag': postag1,  
            '-1:word.istitle()': postag1.istitle(),  
            '-1:postag[:2]': postag1[:2],                
        })  

    else:   

        features['BOS'] = True  

    if i < len(sent)-1:  

        word1 = sent[i+1][0]  
        postag1 = sent[i+1][1]  
        features.update({  

            '+1:word.lower()': word1,  
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isdigit()': word1.isdigit(), 
            '+1:word.isnumeric()': word1.isnumeric(), 
            
            '+1:postag': postag1,   
            '+1:word.istitle()': postag1.istitle(), 
            '+1:postag[:2]': postag1[:2],  
        })  
 

    else:  

        features['EOS'] = True  

    return features  

def sent2features(sent):    
    return [word2features(sent, i) for i in range(len(sent))]  

def sent2labels(sent):  
    return [label for token, postag, label in sent]  

def sent2tokens(sent):  
    return [token for token, postag, label in sent] 

This is what word2features extracts:

In [26]:
sent2features(train[0])[0]

{'bias': 1.0,
 'word.lower()': '記',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isnumeric()': False,
 'postag': 'VK',
 'postag[:2]': 'VK',
 'BOS': True,
 '+1:word.lower()': '得',
 '+1:word.istitle()': False,
 '+1:word.isdigit()': False,
 '+1:word.isnumeric()': False,
 '+1:postag': 'VK',
 '+1:postag[:2]': 'VK'}

Extract features from the data:

In [27]:
%%time
X_train = [sent2features(s) for s in train ]
y_train = [sent2labels(s) for s in train]

X_test = [sent2features(s) for s in validation]
y_test = [sent2labels(s) for s in validation]

Wall time: 2.35 s


#word2vec.vocab['清明'].index

## Training

To see all possible CRF parameters check its docstring. Here we are useing L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [30]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

Wall time: 3min 47s


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

## Evaluation

There is much more O entities in data set, but we're more interested in other entities. To account for this we'll use averaged F1 score computed for all labels except for O. ``sklearn-crfsuite.metrics`` package provides some useful metrics for sequence classification task, including this one.

In [31]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B_Time',
 'I_Time',
 'B_Person',
 'I_Person',
 'B_Thing',
 'I_Thing',
 'B_Location',
 'B_Metric',
 'I_Metric',
 'I_Location',
 'B_Organization',
 'I_Organization',
 'B_Abstract',
 'I_Abstract',
 'B_Physical',
 'I_Physical',
 'B_Term',
 'I_Term',
 'B_ABstract',
 'I_ABstract']

In [32]:
%%time
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

Wall time: 1.29 s


C:\Users\oliver\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\oliver\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.700603315943222

Inspect per-class results in more detail:

In [33]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

C:\Users\oliver\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\oliver\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                precision    recall  f1-score   support

    B_ABstract      0.000     0.000     0.000         0
    I_ABstract      0.000     0.000     0.000         0
    B_Abstract      0.671     0.378     0.483       135
    I_Abstract      0.647     0.437     0.521       591
    B_Location      0.709     0.617     0.660      1826
    I_Location      0.697     0.657     0.676      3237
      B_Metric      0.563     0.467     0.511       304
      I_Metric      0.673     0.644     0.658       626
B_Organization      0.528     0.272     0.359       243
I_Organization      0.588     0.352     0.441       647
      B_Person      0.863     0.857     0.860      4013
      I_Person      0.786     0.793     0.790      3181
    B_Physical      0.000     0.000     0.000         1
    I_Physical      0.000     0.000     0.000         7
        B_Term      0.000     0.000     0.000         0
        I_Term      0.000     0.000     0.000         0
       B_Thing      0.659     0.683     0.671  

# New method using a dictionary

# Method 2

In [58]:
def word2features(sent, i):  
    word = sent[i][0]  
    postag = sent[i][1]  

    punc=[x[1] for x in sent if len(x[1])>6]  
 
    features = {  

        'bias': 1.0,  

        'word.lower()': word,  
        'word.words':words[word],
        'word.istitle()': word.istitle(),  
        'word.isdigit()': word.isdigit(),  
        'word.isnumeric()': word.isnumeric(),   
        
        'postag': postag,  
        'word.istitle()': postag.istitle(), 
        'postag[:2]': postag[:2],   
    }  
 
    if i > 0:  
        
        word1 = sent[i-1][0]  
        postag1 = sent[i-1][1]    
        features.update({  
            
            '-1:word.lower()': word1, 
            '-1:postag': postag1, 
            '-1:word.istitle()': postag1.istitle(),  
            '-1:postag[:2]': postag1[:2],  
            'post.com':tags[postag1]>tags[postag],          
        })  

    else:   

        features['BOS'] = True  

    if i < len(sent)-1:  

        word1 = sent[i+1][0]  

        postag1 = sent[i+1][1]  

        features.update({  

            '+1:word.lower()': word1,
            'compare': words[word1]<words[word],
                
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isdigit()': word1.isdigit(), 
            '+1:word.isnumeric()': word1.isnumeric(), 
           
            '+1:postag': postag1,   
            '+1:postag.tags': tags[postag1],
            '+1:word.istitle()': postag1.istitle(), 
            '+1:postag[:2]': postag1[:2],  
            'post.com2':tags[postag]>tags[postag1],

        })  

    else:  

        features['EOS'] = True  

    return features   

def sent2features(sent):    
    return [word2features(sent, i) for i in range(len(sent))]  

def sent2labels(sent):  
    return [label for token, postag, label in sent]  

def sent2tokens(sent):  
    return [token for token, postag, label in sent] 

In [59]:
words=dict()     
for x in Data:
    for j in x:
        if j[0] not in words:
            words[j[0]]=1/50000
        else:
            words[j[0]]+=1/50000


tags=dict()     
for x in Data:
    for j in x:
        if j[1] not in tags:
            tags[j[1]]=1/300000
        else:
            tags[j[1]]+=1/300000

In [60]:
sent2features(train[0])[0]

{'bias': 1.0,
 'word.lower()': '記',
 'word.words': 0.01829999999999959,
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isnumeric()': False,
 'postag': 'VK',
 'postag[:2]': 'VK',
 'BOS': True,
 '+1:word.lower()': '得',
 'compare': False,
 '+1:word.istitle()': False,
 '+1:word.isdigit()': False,
 '+1:word.isnumeric()': False,
 '+1:postag': 'VK',
 '+1:postag.tags': 0.037703333333338675,
 '+1:postag[:2]': 'VK',
 'post.com2': False}

In [61]:
%%time
X_train = [sent2features(s) for s in train]
y_train = [sent2labels(s) for s in train]

X_test = [sent2features(s) for s in validation]
y_test = [sent2labels(s) for s in validation]

Wall time: 10.8 s


In [63]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

Wall time: 4min 46s


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [64]:
labels = list(crf.classes_)
labels.remove('O')

In [66]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.7009598051992496

In [65]:
%%time
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                precision    recall  f1-score   support

    B_ABstract      0.000     0.000     0.000         0
    I_ABstract      0.000     0.000     0.000         0
    B_Abstract      0.686     0.356     0.468       135
    I_Abstract      0.671     0.408     0.507       591
    B_Location      0.710     0.617     0.660      1826
    I_Location      0.701     0.658     0.679      3237
      B_Metric      0.570     0.470     0.515       304
      I_Metric      0.668     0.644     0.656       626
B_Organization      0.515     0.280     0.363       243
I_Organization      0.574     0.354     0.438       647
      B_Person      0.862     0.858     0.860      4013
      I_Person      0.782     0.795     0.789      3181
    B_Physical      0.000     0.000     0.000         1
    I_Physical      0.000     0.000     0.000         7
        B_Term      0.000     0.000     0.000         0
        I_Term      0.000     0.000     0.000         0
       B_Thing      0.661     0.686     0.673  

# Method 3 ( word2vec)

(word2vec for second self_trained)

In [46]:
def get_real(Dict):
   Real=dict()
   for x in Dict:
    if x not in Real:
        Real[x]=dict()
        for j in range(len(Dict[x])):
            Real[x]['num'+str(j+1)]=Dict[x][j]
   return Real  


In [48]:
%%time
Real=get_real(DDict)

Wall time: 759 ms


In [51]:
def word2features(sent, i):  
    features=Real[sent[i][0] ]
    return features  

def sent2features(sent):  
    return [word2features(sent, i) for i in range(len(sent))]    

def sent2labels(sent):  
    return [label for token, postag, label in sent]  

def sent2tokens(sent):  
    return [token for token, postag, label in sent] 

In [56]:
#sent2features(train[0])[0] 

In [87]:
%%time
X_train = [sent2features(s) for s in train]
y_train = [sent2labels(s) for s in train]

X_test = [sent2features(s) for s in validation]
y_test = [sent2labels(s) for s in validation]

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)
labels = list(crf.classes_)
labels.remove('O')


UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


In [57]:
%%time
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                precision    recall  f1-score   support

    B_ABstract      0.000     0.000     0.000         0
    I_ABstract      0.000     0.000     0.000         0
    B_Abstract      0.810     0.504     0.621       135
    I_Abstract      0.627     0.607     0.617       591
    B_Location      0.469     0.143     0.219      1826
    I_Location      0.441     0.194     0.270      3237
      B_Metric      0.354     0.112     0.170       304
      I_Metric      0.357     0.131     0.192       626
B_Organization      0.256     0.041     0.071       243
I_Organization      0.267     0.164     0.203       647
      B_Person      0.833     0.567     0.675      4013
      I_Person      0.384     0.268     0.316      3181
    B_Physical      0.000     0.000     0.000         1
    I_Physical      0.000     0.000     0.000         7
        B_Term      0.000     0.000     0.000         0
        I_Term      0.000     0.000     0.000         0
       B_Thing      0.327     0.101     0.154  

# Results for the second model

(word2vec for second pre_trained)

In [ ]:
%%time
Real=get_real(DDict2)

%%time
X_train = [sent2features(s) for s in train]
y_train = [sent2labels(s) for s in train]

X_test = [sent2features(s) for s in validation]
y_test = [sent2labels(s) for s in validation]

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)
labels = list(crf.classes_)
labels.remove('O')

In [ ]:
%%time
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

# Word-Based 

In [78]:
validation_word=readdata('validation_word.json')
train_word=readdata('train_word.json')

 Here i create a dictionary to find most common words for each label

In [81]:
%%time
def find_after(train_word,label):
  B=dict()
  for x in train_word:
    for j in range(len(x)-1):
        if x[j][label] not in B:
           B[x[j][label]]=dict()
        f=[x for x in x[j+1][0]]
        for c in f:
            if c not in B[x[j][label]]:
               B[x[j][label]][c]=1
            else:
               B[x[j][label]][c]+=1 
        
  s=cp(B)
  for x in B:
    for j in B[x]:
        B[x][j]/=sum(s[x].values())


  s=cp(B)
  for x in B:
    for j in B[x]:
        B[x][j]/=max(s[x].values())


  N2=dict()

  for x in B:
    N2[x]=[y for y in B[x] if B[x][y]>0.4]

  return N2

Wall time: 0 ns


 feature

In [82]:

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def word2features(sent, i):  

    word = sent[i][0]  
    postag = sent[i][1]  
    c=0.0001
    features = {  

        'bias': 1.0,  
        'word.lower()': word,  
        'word.last':word[-1],
        'word[0]': word[0],  
        'word.b':i==0,
        'word.l':i==len(sent)-1,
        'word.p':np.round(10*(i-len(sent)-1)/(len(sent)-1+c))/10,
        'postag.l': len(postag)>6, 
        'postag[:2]': postag[:2],             
    }  
    if i > 1:  

        word1 = sent[i-1][0]  
        postag1 = sent[i-1][1]  

        features.update({  
                
            '-1:word.lower()': word1,  
            '-1:word.last':word1[-1],
            '-1:word[0]': word1[0],               
            '-1:word.b':i-1==0,
            '-1:word.l':i-1==len(sent)-1,
            '-1.word.p':np.round(10*(i-1-len(sent)-1)/(len(sent)-1+c))/10,
            
            '-1:postag': postag1,
            '-1:postag.l': len(postag1)>6,
            '-1:postag[:2]': postag1[:2],               
        })  

    else:  

        features['BOS'] = True  

    if i < len(sent)-1:
        
        word1 = sent[i+1][0]  
        postag1 = sent[i+1][1]  
        pre_post=[postag, postag1]

        features.update({  
            '+1:word.lower()': word1,  
            '+1:word.last':word1[-1],
            '+1:word[0]': word1[0],  
            '+1:postag': postag1, 
            '+1:postag.l': len(postag1)>6, 
            '+1:postag[:2]': postag1[:2],  

        'B_ABstract.pre_word.a': len(intersection(word1,N2['B_ABstract']))>0,
        'B_Abstract.pre_word.a':len(intersection(word1,N2['B_Abstract']))>0,
        'B_location.pre_word.a':len(intersection(word1,N2['B_Location']))>0,
        'B_Metric.pre_word.a':len(intersection(word1,N2['B_Metric']))>0,
        'B_Organization.pre_word.a':len(intersection(word1,N2['B_Organization']))>0,
        'B_Person.pre_word.a':len(intersection(word1,N2['B_Person']))>0,
        'B_Physical.pre_word.a':len(intersection(word1,N2['B_Physical']))>0,
        'B_Term.pre_word.a':len(intersection(word1,N2['B_Term']))>0,
        'B_Thing.pre_word.a': len(intersection(word1,N2['B_Thing']))>0,
        'B_Time.pre_word.a': len(intersection(word1,N2['B_Time']))>0,
        'I_Abstract.pre_word.a': len(intersection(word1,N2['I_Abstract']))>0,
        'I_location.pre_word.a': len(intersection(word1,N2['I_Location']))>0,
        'I_Metric.pre_word.a': len(intersection(word1,N2['I_Metric']))>0,
        'I_Organization.pre_word.a': len(intersection(word1,N2['I_Organization']))>0,
        'I_Person.pre_word.a': len(intersection(word1,N2['I_Person']))>0,
        'I_Physical.pre_word.a': len(intersection(word1,N2['I_Physical']))>0,
        'I_Term.pre_word.a': len(intersection(word1,N2['I_Term']))>0,
        'I_Thing.pre_word.a': len(intersection(word1,N2['I_Thing']))>0,
        'I_Time.pre_word.a':len(intersection(word1,N2['I_Time']))>0,
        'O.pre_word.a':len(intersection(word1,N2['O']))>0, 
        })    
    else:  
        features['EOS'] = True  
    return features  

def sent2features(sent):  
    return [word2features(sent, i) for i in range(len(sent))]  

def sent2labels(sent):  
    return [label for token, postag, label in sent]  

def sent2tokens(sent):  
    return [token for token, postag, label in sent] 

In [83]:
%%time
N2=find_after(train_word,2)

In [84]:
%%time
X_train = [sent2features(s) for s in train_word]
y_train = [sent2labels(s) for s in train_word]

X_test = [sent2features(s) for s in validation_word]
y_test = [sent2labels(s) for s in validation_word]

In [85]:
%%time
crf= sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)


crf.fit(X_train, y_train)

labels = list(crf.classes_)
labels.remove('O')


y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

C:\Users\oliver\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\oliver\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Wall time: 5min 27s


0.7139510606113944

In [86]:
%%time
sorted_labels= sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
 y_test, y_pred, labels=sorted_labels, digits=3
))

C:\Users\oliver\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\oliver\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                precision    recall  f1-score   support

    B_ABstract      0.000     0.000     0.000         0
    B_Abstract      0.707     0.430     0.535       135
    I_Abstract      0.737     0.512     0.604       322
    B_Location      0.700     0.626     0.661      1826
    I_Location      0.654     0.588     0.619      1233
      B_Metric      0.565     0.487     0.523       304
      I_Metric      0.591     0.519     0.552       351
B_Organization      0.500     0.300     0.375       243
I_Organization      0.596     0.363     0.451       248
      B_Person      0.895     0.902     0.898      4013
      I_Person      0.682     0.639     0.660       728
    B_Physical      0.000     0.000     0.000         1
    I_Physical      0.000     0.000     0.000         5
        B_Term      0.000     0.000     0.000         0
        I_Term      0.000     0.000     0.000         0
       B_Thing      0.707     0.758     0.731      3144
       I_Thing      0.466     0.526     0.494  